# Model inference

This notebook shows how to load the trained model for inference. This includes processing new data to prepare it for inference.

> This process will likely be captured in a Docker Repo asset on Highwind

In [1]:
# Config
ARTIFACT_SAVE_DIR = "../saved_model/"

# For this example, the test data will be used for inference
TEST_DATA_PATH = "../data/test.csv"
TARGET_COLUMN = "MedHouseVal"

In [2]:
import os
import pandas as pd
import joblib
from huggingface_hub import hf_hub_download

/Users/ruan/Documents/github/highwind-examples/california-housing/.serve.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load data

In [3]:
test_df = pd.read_csv(TEST_DATA_PATH)
test_df.head(3)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,1.6812,25.0,4.192201,1.022284,1392.0,3.877437,36.06,-119.01,0.47700
1,2.5313,30.0,5.039384,1.193493,1565.0,2.679795,35.14,-119.46,0.45800
2,3.4801,52.0,3.977155,1.185877,1310.0,1.360332,37.80,-122.44,5.00001


In [4]:
# Separate features and labels
X_test = test_df.copy()
y_test = X_test.pop(TARGET_COLUMN)

In [5]:
# Check shapes
print(f"X_test: {X_test.shape}")
print(f"y_test: {y_test.shape}")

X_test: (4128, 8)
y_test: (4128,)


## Download trined model and feature scaler

In [6]:
# Download from Hugging Face
download_files = ["model.joblib", "scaler.joblib"]

for file_name in download_files:
    hf_hub_download(
        repo_id="MelioAI/california-housing",
        filename=file_name,
        local_dir=ARTIFACT_SAVE_DIR
    )

## Process data

Repeat all processing steps to prepare data for inference

In [7]:
# Feature scaling
scaler = joblib.load(os.path.join(ARTIFACT_SAVE_DIR, "scaler.joblib"))
X_test_scaled = scaler.transform(X_test)

## Make predictions

In [8]:
# Load model
model = joblib.load(os.path.join(ARTIFACT_SAVE_DIR, "model.joblib"))
model

Lasso(alpha=0.01, random_state=42)

In [9]:
# Make predictions on processed data
y_pred = model.predict(X_test_scaled)
y_pred

array([0.78787312, 1.73253525, 2.5649099 , ..., 4.40562063, 1.2930991 ,
       1.95300562])